Library importation

In [ ]:
import numpy as np
import pandas as pd
import mathplotlib as plt
from pathlib import Path
from datetime import datetime

Function definition

In [ ]:
def dparser(d,t):
    return pd.to_datetime(f'{d} {t}',format='%Y/%m/%d %H:%M:%S')
def vega_reader(file):
    print(f" ... load file {file}")
    df=pd.read_csv(file,comment='#', names=['Date','Time','Vega','Source'], parse_dates={'datetime':[0,1]}, 
                   delim_whitespace=True,date_parser=dparser)
    df.set_index('datetime',inplace=True)
    df.drop(columns=['Source'],inplace=True)
    return df

def vdc_test(R, I, ref='REFERENCE', inst='INSTRUMENT'):
   
    vdc=pd.concat([R,I],axis=1)
    vdc['difference']=vdc[ref]-vdc[inst] # add a 'diff' column 
    ax=vdc.plot(x='difference',y=inst,marker='.',title='VDC: '+ref+' - '+inst)
    ax.axvline(0,color='red')
    plt.ylabel('Sea level (m)')
    ax.grid()

def running_mean(df,input_col, t, output_col,frequency,manipulate_time=True):  #col="column_name", manipulate time = true or false
    """t is the integration period,input_col= data column, output_col=name of the new column
    frequency is the desired sampling rate of the data,
    if we manipulate the time then the index will be uniform minute with second zero
    """
    int_period = pd.Timedelta(seconds=t)
    if manipulate_time:
            index_range = pd.date_range(start='2021-09-28 09:50:00', end='2021-09-29 07:00:00', freq=frequency)
    else:        
            index_range = pd.date_range(start='2021-09-28 09:52:52', end='2021-09-29 06:52:52', freq=frequency)
    
    sl = np.zeros(len(index_range))
    
    for i in range(len(index_range)):
        sl[i] = df[index_range[i]-int_period:index_range[i]+int_period][input_col].mean()
        
    df_out = pd.DataFrame(sl, index=index_range, columns=[output_col])
    return df_out

The measurement done by the Vega Radar tide gauge are not given to any reference. How ever it's of the out most important that those data are, otherwise they are meaningless. Thus we need to express the position of our Radar with respect to the hydrographic zero and, in a second time to the reference ellipsoïde GRS80ellipsoid in the RGF93 geocentric reference frame.

Three session of leveling has been made. the fisrt one conscidering the "Marche Arrière" to be the benchmark F and the "Marche Avant" to the the Vega RADAR, while the 2 last one "Marche Arrière" and "Marche Avant" have been switch. The RADAR elvation will be the mean of the elevations given by thos three measurment.

The first step will be to give the elevationof our instrument with respect to a near by Benchmark(F).
How ever a problem arise when we take in acount that  there is a constant offset of 0.019 m between the spot were the radar do the measurment and the spot where the ruler was put to make the leveling measurment.As suche the real Elevation(D) of the radar would be the Measured Elevation minus this offset. 

D=-0.0188 m

<img src="https://i.imgur.com/KGEflwg.png"/>

If we go bach to the definition of the elevation D :
D=Ar-Av <=> D = Hav-Har = Hv-Hf
then if D < 0 this mean that Hv < Hf ie the RADAR is bellow the benchmark.

The benchmark can then be linked to the hydrographic zero and the reference ellipsoid GRS80 of the RGF93.

the produced off set need to be taken in account in order to express our data in a meaningfull reference system. Then they can be properly compared to the SHOM data for Aix Island.
As Stated earlier we determine the elevation of our Tide Gauge with respect to the Benchmark F. The ladder being link to the rest of the reference Benchmark, as shown in the Tide observation paper oh the SHOM. By drawing a "path" from our tide gauge to the reference we would like to use we can determine it's elevation with respect to that reference.

<img src="https://i.imgur.com/xdi47ML.png"/>

Since our VEGA was 0.0188 m bellow the Benchmark, for our measure to be at the same level than the benchmark we need to ADD this eleveation. But since F is above the B benchmark we need to SUBTRACT the elevation for ou measurment to be at the same level than B. Thus we can devise a "path" between the diferent benchmark to determine the Tide Gauge elevation 
with respecte to the two references, like so :

Vega=>F=>B=>hydrographic zero

Vega=>F=>B=>P=>R=>Q=>GRS80

In [4]:
#offset to the hydrographic zero and reference ellipsoïde GRS80
"""I'm not sure if this part should stay in the final for m of the porject. it's fairly simple but can be used as 
a start for a function that put the offset to every data we got. i'll put a figure to explain 
the logic behind those offset """
D=-0.0188
VF=-D
FB=-0.198
BZh=-7.650
BP=0.435
PR=4.257
RQ=3.370
QGRS80=-59.027
VZh=VF+FB+BZh
print("Hydrographic elevation",VZh,"m" )
VGRS80=VF+FB+BP+PR+RQ+QGRS80
print("GRS80 elevation",VGRS80,"m" )

Hydrographic elevation -7.8292 m
GRS80 elevation -51.1442 m


To exprime the Sea Level within those reference frame it's only a matter of applying those correction. It is impereative that the Hydrographic zero correction is applied to the sea level data before the comparaison to the SHOM data for Aix Iland. Other whise the comparaison is meaningless.